In [1]:
import json
import requests
import praw
import numpy

reddit = praw.Reddit(client_id='tiBit4kBeuoOTA', 
                     client_secret='7-HPDa9q92MS0a-AnUVYb72QDzU', 
                     user_agent='TorkilSamStat', 
                     username='TorkSamStat', 
                     password='bbybbyduerinnafor')

# STATIC VARIABLES
base_url = 'https://api.pushshift.io/reddit/search/comment/'
wanted_count_auth = 2
wanted_count_sub = 10
unwanted_users = ["[deleted]", "AutoModerator"]
unwanted_count = len(unwanted_users)
used_authors = []
used_subreddits = []
offset_limit = 4
    
# FUNCTIONS
def getKeysFromDict(data):
    users = []
    for obj in data:
        if obj['key'] not in unwanted_users:
            users.append(obj['key'])
    return users

def getMostActiveAuthorsInSubReddit(subreddit):
    if subreddit in used_subreddits:
        return []
    else:
        url = base_url + '?since=100d&subreddit={}&aggs=author&agg_size={}'.format(subreddit, wanted_count_auth + unwanted_count)
        response = requests.get(url).json()
        fetched_authors = getKeysFromDict(response['aggs']['author'])
        return fetched_authors

def getMostActiveSubRedditForAuthor(author):
    if author in used_authors:
        return []
    else:
        url = base_url + '?since=100d&author={}&aggs=subreddit&agg_size={}'.format(author, wanted_count_sub)
        response = requests.get(url).json()
        fetched_subreddits = getKeysFromDict(response['aggs']['subreddit'])
        for sub in used_subreddits:
                if (sub not in used_subreddits):
                    used_subreddits.append(sub)
        return fetched_subreddits

In [2]:
relation_array = []
users_1 = getMostActiveAuthorsInSubReddit('The_Donald')
used_subreddits.append("The_Donald")

subcount = {}
for user in users_1:
    subs = getMostActiveSubRedditForAuthor(user)
    for sub in subs:
        if sub not in list(subcount.keys()):
            subcount[sub] = 1
        else:
            subcount[sub] += 1
        relation_array.append(['The_Donald', sub])
        # NIVÅ 2 START
        users_2= getMostActiveAuthorsInSubReddit(sub)
        for user2 in users_2:
            subs2 = getMostActiveSubRedditForAuthor(user)
            for sub2 in subs2:
                if sub2 not in list(subcount.keys()):
                    subcount[sub2] = 1
                else:
                    subcount[sub2] += 1
                relation_array.append([sub, sub2])
                
                used_subreddits.append(sub)
            used_authors.append(user2)
        # NIVÅ 2 SLUTT
        
        
        
        
        
        
    used_authors.append(user)


relation_array

NameError: name 'user2' is not defined

In [ ]:
print(subcount)

### 